In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [0]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [0]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [0]:
sen_df = spark.createDataFrame([
  (0, 'Hi I heard about spark'),
  (1, 'I wish java could use case classes'),
  (2, 'logistic,regression,models,are,neat')
],['id','sentence'])

In [0]:
sen_df.show()

+---+--------------------+
 id| sentence|
+---+--------------------+
 0|Hi I heard about ...|
 1|I wish java could...|
 2|logistic,regressi...|
+---+--------------------+

##### word tokenize

In [0]:
tokenizer = Tokenizer(inputCol = 'sentence', outputCol = 'words')

In [0]:
regex_tokenizer = RegexTokenizer(inputCol = 'sentence', outputCol = 'words', pattern = '\\W')
# \W match any character which is not word character, equals to [^a-zA-Z0-9_]

In [0]:
count_tokens = udf(lambda words:len(words), IntegerType())

In [0]:
tokenized = tokenizer.transform(sen_df)

In [0]:
tokenized.show()

+---+--------------------+--------------------+
 id| sentence| words|
+---+--------------------+--------------------+
 0|Hi I heard about ...|[hi, i, heard, ab...|
 1|I wish java could...|[i, wish, java, c...|
 2|logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+

In [0]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
 id| sentence| words|tokens|
+---+--------------------+--------------------+------+
 0|Hi I heard about ...|[hi, i, heard, ab...| 5|
 1|I wish java could...|[i, wish, java, c...| 7|
 2|logistic,regressi...|[logistic,regress...| 1|
+---+--------------------+--------------------+------+

In [0]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [0]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
 id| sentence| words|tokens|
+---+--------------------+--------------------+------+
 0|Hi I heard about ...|[hi, i, heard, ab...| 5|
 1|I wish java could...|[i, wish, java, c...| 7|
 2|logistic,regressi...|[logistic, regres...| 5|
+---+--------------------+--------------------+------+

##### remove stop words

In [0]:
from pyspark.ml.feature import StopWordsRemover

In [0]:
sentenceDF = rg_tokenized.select('id','words')

In [0]:
remover = StopWordsRemover(inputCol = 'words', outputCol = 'filtered')

In [0]:
remover.transform(sentenceDF).show()

+---+--------------------+--------------------+
 id| words| filtered|
+---+--------------------+--------------------+
 0|[hi, i, heard, ab...| [hi, heard, spark]|
 1|[i, wish, java, c...|[wish, java, use,...|
 2|[logistic, regres...|[logistic, regres...|
+---+--------------------+--------------------+

#### n-grams

In [0]:
from pyspark.ml.feature import NGram

In [0]:
ngram = NGram(n = 2, inputCol = 'words', outputCol = 'grams')

In [0]:
ngram.transform(sentenceDF).show(truncate = False)

+---+------------------------------------------+------------------------------------------------------------------+
id |words |grams |
+---+------------------------------------------+------------------------------------------------------------------+
0 |[hi, i, heard, about, spark] |[hi i, i heard, heard about, about spark] |
1 |[i, wish, java, could, use, case, classes]|[i wish, wish java, java could, could use, use case, case classes]|
2 |[logistic, regression, models, are, neat] |[logistic regression, regression models, models are, are neat] |
+---+------------------------------------------+------------------------------------------------------------------+

#### TF-IDF

In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [0]:
sentence = spark.createDataFrame([
  (0.0, 'Hi I heard about spark'),
  (0.0, 'I wish Java could use case classes'),
  (1.0, 'Logistic regression models are neat')
], ['label','sentence'])

In [0]:
sentence.show()

+-----+--------------------+
label| sentence|
+-----+--------------------+
 0.0|Hi I heard about ...|
 0.0|I wish Java could...|
 1.0|Logistic regressi...|
+-----+--------------------+

In [0]:
tokenizer = Tokenizer(inputCol = 'sentence', outputCol = 'words')

In [0]:
words_data = tokenizer.transform(sentence)

In [0]:
words_data.show()

+-----+--------------------+--------------------+
label| sentence| words|
+-----+--------------------+--------------------+
 0.0|Hi I heard about ...|[hi, i, heard, ab...|
 0.0|I wish Java could...|[i, wish, java, c...|
 1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+

In [0]:
hashing_tf = HashingTF(inputCol = 'words', outputCol = 'rawFeatures')

In [0]:
featurized_data = hashing_tf.transform(words_data)

In [0]:
idf = IDF(inputCol = 'rawFeatures', outputCol = 'features')

In [0]:
idf_model = idf.fit(featurized_data)

In [0]:
rescaled_data = idf_model.transform(featurized_data)

In [0]:
rescaled_data.show()

+-----+--------------------+--------------------+--------------------+--------------------+
label| sentence| words| rawFeatures| features|
+-----+--------------------+--------------------+--------------------+--------------------+
 0.0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[18700,19...|(262144,[18700,19...|
 0.0|I wish Java could...|[i, wish, java, c...|(262144,[19036,20...|(262144,[19036,20...|
 1.0|Logistic regressi...|[logistic, regres...|(262144,[46243,58...|(262144,[46243,58...|
+-----+--------------------+--------------------+--------------------+--------------------+

#### count vectorizer

In [0]:
from pyspark.ml.feature import CountVectorizer

In [0]:
df = spark.createDataFrame([
  (0, 'a b c'.split(' ')),
  (1, 'a b b c a'.split(' '))
], ['id', 'words'])

In [0]:
df.show()

+---+---------------+
 id| words|
+---+---------------+
 0| [a, b, c]|
 1|[a, b, b, c, a]|
+---+---------------+

In [0]:
cv = CountVectorizer(inputCol = 'words', outputCol = 'features', vocabSize = 3, minDF = 2.0)

In [0]:
model = cv.fit(df)

In [0]:
model.transform(df).show(truncate = False)

+---+---------------+-------------------------+
id |words |features |
+---+---------------+-------------------------+
0 |[a, b, c] |(3,[0,1,2],[1.0,1.0,1.0])|
1 |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

#### nlp code along project

##### create spark session

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlpproj').getOrCreate()

##### load data

In [0]:
path = '/FileStore/tables/SMSSpamCollection'
data = spark.read.csv(path, inferSchema = True, sep = '\t')

In [0]:
data.show()

+----+--------------------+
 _c0| _c1|
+----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
spam|WINNER!! As a val...|
spam|Had your mobile 1...|
 ham|I'm gonna be home...|
spam|SIX chances to wi...|
spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows

##### rename data columns

In [0]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [0]:
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

##### check text length

In [0]:
from pyspark.sql.functions import length

In [0]:
data = data.withColumn('length', length(data['text']))

In [0]:
data.groupBy('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

##### feature engineering

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [0]:
tokenizer = Tokenizer(inputCol = 'text', outputCol = 'token_text')
stop_remover = StopWordsRemover(inputCol = 'token_text', outputCol = 'stop_tokens')
cv = CountVectorizer(inputCol = 'stop_tokens', outputCol = 'c_vec')
idf = IDF(inputCol = 'c_vec', outputCol = 'tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol = 'class', outputCol = 'Label')

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
clean_up = VectorAssembler(inputCols = ['tf_idf', 'length'], outputCol = 'features')

##### create data preprocessing pipeline

In [0]:
from pyspark.ml.classification import NaiveBayes

In [0]:
nb = NaiveBayes()

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_prep_pipe = Pipeline(stages = [ham_spam_to_numeric, tokenizer, stop_remover, cv, idf, clean_up])

In [0]:
cleaner = data_prep_pipe.fit(data)

In [0]:
clean_data = cleaner.transform(data)

In [0]:
clean_data = clean_data.select('features','label')

##### split data into training set and test set

In [0]:
training, test = clean_data.randomSplit([0.7, 0.3])

##### modeling

In [0]:
spam_detector = nb.fit(training)

In [0]:
test_results = spam_detector.transform(test)

In [0]:
test_results.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|label| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
(13424,[0,1,2,5,5...| 1.0|[-937.78220395686...|[0.99999999999924...| 0.0|
(13424,[0,1,2,7,8...| 0.0|[-792.16012052336...|[1.0,7.7884261933...| 0.0|
(13424,[0,1,2,12,...| 1.0|[-1135.8106205503...|[1.75803854208698...| 1.0|
(13424,[0,1,2,15,...| 1.0|[-1172.0047677349...|[5.87015452831360...| 1.0|
(13424,[0,1,2,15,...| 1.0|[-1155.6558361441...|[3.29534622603644...| 1.0|
(13424,[0,1,2,20,...| 1.0|[-1309.1192616328...|[1.13244935286266...| 1.0|
(13424,[0,1,2,41,...| 0.0|[-1058.2994225988...|[1.0,5.9377937945...| 0.0|
(13424,[0,1,2,47,...| 1.0|[-1494.8533978182...|[1.15261357190307...| 1.0|
(13424,[0,1,2,91,...| 1.0|[-1012.5238460133...|[2.21021362772756...| 1.0|
(13424,[0,1,4,50,...| 0.0|[-851.48425906312...|[1.0,2.4865009632...| 0.0|
(13424,[0,1,5,15,...| 0.0|[-999.13153316796...|[1.0,1.5432182652...| 0.0|
(13424,[0,1,7,15,...| 0.0|[-660.08612894099...|[1.0,9.8536117962...| 0.0|
(13424,[0,1,9,14,...| 0.0|[-540.43529260113...|[1.0,1.6364880302...| 0.0|
(13424,[0,1,11,32...| 0.0|[-867.18272861723...|[1.0,2.9101577538...| 0.0|
(13424,[0,1,14,78...| 0.0|[-687.31357724491...|[1.0,1.0045523152...| 0.0|
(13424,[0,1,21,27...| 0.0|[-771.59707381300...|[1.0,2.0741578125...| 0.0|
(13424,[0,1,30,12...| 0.0|[-599.01267308632...|[1.0,1.0837253693...| 0.0|
(13424,[0,1,31,43...| 0.0|[-343.58596281772...|[1.0,6.0436659332...| 0.0|
(13424,[0,1,72,10...| 0.0|[-683.28963816479...|[1.0,1.0058165860...| 0.0|
(13424,[0,1,98,18...| 1.0|[-1635.7624519935...|[4.77445469971057...| 1.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

##### evaluation

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
acc_eval =  MulticlassClassificationEvaluator()

In [0]:
acc = acc_eval.evaluate(test_results)

In [0]:
print('ACC of NB model')
print(acc)

ACC of NB model
0.923289052759044